# 4th Down Bot

The NYT 4th Down Bot uses NFL play-by-play data to make a recommendation for 4th down decisions.  The current model behind the 4th Down Bot estimates the win probability given a state of the game.  We will do a recreation of the original 4th Down Bot model from Brian Burke (now of ESPN, formerly of Advanced Football Analytics).

The basic premise of the model is to maximize the expected value of the 4th down decision. To do this, we will use NFL play-by-play data to estimate quantities related to the value of a location on the field, punt distance, likelihood of converting a first down, and likelihood of making a field goal.  This notebook builds a replica of the bot.


## The 4th Down Decision Making Model

Our goal will be to maximize the expected value of the decision to go for it, punt, or kick a field goal.  This means we need to compute the following four core models:

1. **Expected Possession Value (EPV):** The value, in points, of a first down at any location on the field.  Kickoffs will also require an expected value.
2. **Expected net punt distance:** Net punt distance given a location on the field.  Coupled with EPV, this quantifies the expected value of punting.
3. **Probability of making a 4th down conversion:** Probability of conversion given a location on the field, eg. how likely a team is to make a 4th and 3 conversion from the opponent 47 yard line.  We can use this with EPV to quantify the expected value of going for the first down.
4. **Field Goal Probability:** Probability of making a field goal from a given distance, eg. how likely it is to make a 45 yard field goal.  Combined with the EPV for field location and kickoffs, we can compute an expected value of kicking the field goal by estimating the likelihood and expected value of a miss or a make.
 
These 4 quantities can be combined to produce expected value of going for the first down, punting, or kicking the field goal.  Our decision rule will be to choose the option that maximizes expected value.  The model we will build is a baseline "average" model that can be used as a starting point before factoring in issues like quality of field goal kicker, offense or defense quality, or weather.


## Tasks

More detail on the individual computations is below.

There are a few code elements to fill in towards the end.  Predominantly there are questions around the modeling process as well as questions about decision making in general at th end.



_References on the 4th Down Bot and 4th down decision making in general_

+ The NYT 4th Down Bot can be found here: http://nyt4thdownbot.com/ 
+ The model: http://www.advancedfootballanalytics.com/index.php/home/research/game-strategy/120-4th-down-study
+ Coverage: http://www.nytimes.com/newsgraphics/2013/11/28/fourth-downs/post.html
+ More coverage: https://www.nytimes.com/2015/10/02/upshot/a-better-4th-down-bot-giving-analysis-before-the-play.html
+ Slides on the construction of the new model: http://slides.com/treycausey/4thdownbot#/
+ The new model on GitHub: https://github.com/TheUpshot/4thdownbot-model
+ Berkeley's own David Romer on 4th down decisions: https://eml.berkeley.edu/~dromer/papers/JPE_April06.pdf



In [ ]:
%run ../../utils/notebook_setup.py

In [ ]:
import numpy as np

from datascience import Table, table_apply, are

from datascience_stats import curve_fit
from datascience_topic import *

## NFL PxP Data

NFL play-by-play data is available in csv format.  The required data for building the model has already been computed.  See the companion notebooks if you want to know more about how the data was handled to aid this assignment. 

Below is a partial list of the available columns in the full play-by-play dataset.  Many are self explanatory so when needed, a description will be given.  These are the main columns that get used to put together a data analysis.  There are many more fields available that are not used.
+ GameID
+ Drive - index given the # of the drive within the game
+ Quarter
+ Down
+ Yardline100 - the yard line expressed on a scale of 1 to 99 instead of 1 to 50 and back to 1.
+ YardstoGo - yards to go for a first down
+ Yards.Gained - yards gained on the play
+ PossessionTeam - possessing team
+ DefensiveTeam - defensive team
+ PlayType - label for what type of play
+ Touchdown - 0,1 indicating if a TD was scored
+ FieldGoalResult - label indicating good, blocked, or no good.
+ FieldGoalDistance
+ PosTeamScore - Score of the possessing team.  This will flip when the possession flips.
+ DefTeamScore - Score of the defensive team.  This will flip when the possession flips.
+ HomeTeam
+ AwayTeam

A few convenient data fields are added to easy computation of possession value.

+ Half
+ YrdRegion - region of the field: Inside the 10, 10 to 20, and beyond 20.
+ HomeScore & AwayScore - The score of the possession and defensive teams are given.  This changes as the ball changes possession
+ NextPossessionTeam - The team possessing the ball in the next play. Non-plays are ignored
+ NextYardline100 - Where the ball is on the next play. Non-plays are ignored
+ NextDown - The down for the next play
+ 1stDownConversion - Whether the current play converted a first down (0 or 1 value)

In [ ]:
pxp = Table.read_table('data/nfl_pxp_2009_2016.csv.gz', sep=',', nrows=15)
pxp.show()

## Model 1: Expected Possession Value for Kickoffs and First Downs

From the full play-by-play, we need kickoffs and first downs in order to build a possession value calculator.  To do that, this process was used:
1. **Extract kickoffs by using PlayType.**
2. **Extract all first downs**  Drill down by GameID and take all first downs that have 10 yards to go or goal to go.  
3. **Find the next score in the game for each possession.**  This is the hardest computation.  We do this by computing differences in the home and away scores and then fill those backward.  We treat home scores as positive and away scores as negative.  We only consider possession value within a half.  So if there is no score before halftime or the end of the game, the value is 0.   Here is a toy example:
    
| Drive | Team | Poss Score | Next Score |
| ----- | ---- | ---------- | ---------- |
| 1     | Pit  | 0          |   Pit +7   |
| 2     | NYJ  | 0          |   Pit +7   |
| 3     | Pit  | 7          |   Pit +7   |
| 4     | NYJ  | 3          |   NYJ +3   |

4. **Compute possession value.**  We multiply the next score value by +1 or -1 depending on if the current possessing team is the home team or away team.  If its the home team, then multiply by +1 because the next score is already oriented to the home team.  If its the away team, then multiply by -1 because a positive next score is a negative for the away team.
5. **Restrict possessions to the first and third quarter.**

Some caveats:
+ The dataset used is not perfect so while we expect this procedure to work a vast majority of the time, it may miss some results because of holes in the dataset.  It is unlikely this affects the analysis too much.
+ While we restricted to first and third quarters, we did not restrict cases when there is a blowout.  Competitive games lead to more reliable results so this is probably the first issue to address going forward.

The kickoff and first down data can be loaded in `data/nfl_first_downs_2009_2016.csv.gz`.

In [ ]:
states_1st_downs_kickoffs = Table.read_table('data/nfl_first_downs_2009_2016.csv.gz', sep=',')
states_1st_downs_kickoffs.show(10)

**Questions**

1. Why do we want to only use data from the first and third quarters?  
2. We didn't restrict to non-blowouts situations but how would removing blowouts help clean up the analysis?  
3. In general, why do we remove certain observations from a dataset?

**Answers**

### A. Expected Kickoff Value

In the dataset, the return team is considered the possessing team.  So computing the average value represents the expected value to the return team.

After this, `ekv` contains our model for Expected Kickoff Value.

In [ ]:
# Restrict to kickoff events
kickoffs = states_1st_downs_kickoffs.where('PossessionType', 'Kickoff')

# Compute the average kickoff value
ekv = kickoffs['PossessionValue'].mean()
print(f"Expected Kickoff Value: {ekv:.3f} pts")

### B. Expected Possession Value 

We group by each yard line, excluding kickoffs, and compute the average possession value.

The plot below shows the results of our estimation.  We clearly see a strong relationship between the Raw Expected Possession Value but the noise in the estimation is apparent.  The expected value should be "smooth" and increasing as you get closer and not fluctuating so much.

In [ ]:
first_downs = states_1st_downs_kickoffs.where('PossessionType', 'FirstDown')

In [ ]:
# 1. Ignore kickoffs
# 2. Group by yardline 
# 3. Take the average of PossessionValue
epv = first_downs.\
    select('Yardline100', 'PossessionValue').\
    group('Yardline100', collect=np.mean).\
    relabel('PossessionValue mean', 'rawEPV')
epv

In [ ]:
epv.scatter('Yardline100', select='rawEPV', height=5, width=8)

### C. Smoothed Expected Possession Value Model


#### Model Smoothing

Suppose we have data that is noisy but we know the underlying model should be smooth.  We can do some basic curve fitting, or _smoothing_, to clean up the data and give us something nice.  The next cell shows a simple example of this using the provided function `curve_fit`.  The parameter `smoothness` controls how smooth the output is.  Feel free to vary the parameter to see how the output changes.

In [ ]:
x = np.linspace(-1, 1)
y = x**2 + .2 * np.random.randn(len(x)) # y is a noisy parabola

fig, ax = plt.subplots(figsize=(8, 5))
ax.plot(x, y);

y_smooth = curve_fit(x, y, smoothness=.5)
ax.plot(x, y_smooth);

#### Smoothing Expected Possession Value

We can now use the curve fitting to produce a "smoothed" estimate of the Expected Possession Value.

After this, `epv_model` contains our model for Expected Possession Value for a location on the field.

In [ ]:
yardlines = epv['Yardline100']
poss_vals = epv['rawEPV']

# create the smoothed EPV values using a custom smoothness value
epv_values = curve_fit(yardlines, poss_vals, smoothness=.22)
# create the EPV model
epv['EPV'] = epv_values

In [ ]:
# fig, ax =  plt.subplots(figsize=(8, 5))
epv.scatter('Yardline100', select='rawEPV', height=5, width=8)

ax = plt.gca()
ax.plot(epv['Yardline100'], epv['EPV'], lw=3);

#### Make the Model

To make the model useable, we need to convert it to a dictionary.  The next cell shows how this is done.

In [ ]:
def make_model(keys, values):
    model = {key: val for key, val in zip(keys, values)}
    return model

epv_model = make_model(epv['Yardline100'], epv['EPV'])

**Questions**

1. Around the 10 yard line (90 in Yardline100), the expected value is 0.  What does this mean?
2. What is the significance of the slope of the line?  What are its units?
3. There appears to be an elbow in the curve around the opponent 10 yard line (10 in Yardline100).  This is where first downs go from 10 yards to go to goal to go.  Can you provide a possible reason for this increase in the rate of EPV?
4. Consider two hypothetical alternative models for expected point:
    + The line goes from left to right from +6 to +5 (much less steep than the actual model).  
    + The line goes from +6 to -6 and passes 0 at the 50 yard line (much steeper than the actual model).
   
   What would you conclude about possession and scoring in these two hypothetical situations?


**Answers**

## Model 2: Net Punt Distance

We need net punt distance.  Extracting punts from the full play-by-play is easy (there's a playtype flag).  Determining net punt distance is tricky.  A few things can happen on a punt: 
+ It goes off as expected
+ Returned for a touchdown
+ Muffed
+ Fumbled
+ Blocked.  

There may be even more wrinkles.  Suffice to say, this is a bit complicated.  To simplify things, we just want to know the expected net punt distance for punts that are not returned, muffed, fumbled, blocked, or anything else.  

We compute net punt distance by finding regular punts and determining the field position change.

In [ ]:
punts = Table.read_table('data/nfl_punts_2009_2016.csv', sep=',')
punts.show()

**Question**

1. Why is it okay to simplify things?  To assist in reasoning out your response, it's helpful to know that in 2017 NFL teams averages 1 punt return TD per 100 punt return attempts.  The key question of whether it's okay to simplify things is this: By about how many points in expectation does ignoring punt return TDs change the result?

**Answer**

### A. Net Punt Distance

To compute net punt distance, we consider all the "successful" punts we extracted (ie. ignoring return TDs, muffs, and fumbles, etc).  In building the punts table, we look at the current yardline and the next yardline and computed a difference (accounting for the switch in orientation of the 100 yard value) to get net punt distance.  Now we just need to average.

In the plot below, we see a beautiful relationship between expected punt distance and the yardline.  As with Expected Possession Value, the expected net punt distance should be increasing as you get closer and not fluctuating as much.

#### TO DO

In [ ]:
# 1. Select the columns Yardline100 and NetPuntDist
# 2. Group by Yardline100
# 3. Take the average of NetPuntDist
punt_dist = ...

In [ ]:
# clean up table name
punt_dist = punt_dist.relabel('NetPuntDist mean', 'rawNetPuntDist')
punt_dist

In [ ]:
punt_dist.scatter('Yardline100', select='rawNetPuntDist', height=5, width=8)

ax = plt.gca()
ax.set_xlim(-.3, 102.3)
ax.set_ylim(-4, 53);

### C. Smoothed Net Punt Distance

We can run our curve fit to generate a final model.  We clearly require this since we do not want a straight line for Expected Net Punt Distance.

`punt_dist_model` will contain our model for Expected Net Punt Distance.

#### TO DO

In [ ]:
yards = punt_dist['Yardline100']
raw_punt_dist = punt_dist['rawNetPuntDist']

# Fit a curve for punt distance using curve_fit
# See the usage of curve_fit in the section on the expected possession value
# use the default smoothness parameter
xpunt_dist = ...

# Convert to integer values (this will be needed later)
xpunt_dist = np.round(xpunt_dist).astype(int)
punt_dist['NetPuntDist'] = xpunt_dist

In [ ]:
punt_dist.scatter('Yardline100', select='rawNetPuntDist', height=5, width=8)

ax = plt.gca()
ax.plot(punt_dist['Yardline100'], punt_dist['NetPuntDist'], lw=3)

ax.set_xlim(-.3, 102.3)
ax.set_ylim(-4, 53);

#### Make the Model

In [ ]:
punt_dist_model = make_model(punt_dist['Yardline100'], punt_dist['NetPuntDist'])

**Question**
1. Without redoing everything, what is one way you could tweak the model to account for the quality of the punter or the quality of the punt coverage by the punting team?

**Answer**

## Model 3: 1st Down Conversion Probability


In football, teams typically treat third down as do or die.  Hence, we are going to use third down conversion percentage as a proxy for 4th down conversion percentage.  You shouldn't have a hard time convincing yourself why it make sense to approximate the conversion percentage using 3rd down data instead of 4th down data.

There are some things we need to address.  In the dataset, various non-relevant plays can occur on third down.  We need to ignore these.  Also, we can likely safely assume that beyond 9 yards we do not need to consider going for it.

In [ ]:
third_downs = Table.read_table('data/nfl_third_downs_2009_2016.csv', sep=',')
third_downs.show(10)

### A. Expected Conversion Percentage

#### Conversion Percentage by Region

We each separate region on the field (inside the 10 yardline, from the 10 yardline to the 20 yardline, and beyond the 20 yardline), and compute the expected conversion percentage for each yard to go value.  We separate into regions because we have reason to believe that near the goalline, the offense is less likely to convert given less room to work with.  The data bears this out.  This change in conversion percentage can affect the expected value quantities.

As with possession value, we need to smooth out the values since they should be consistent as the yards change incrementally.

In [ ]:
# 1. Select the columns YrdRegion, YardstoGo, 1stDownConversion
# 2. Group by YrdRegion and YardstoGo
# 3. Take the average on 1stDownConversion.
conv_pct = ...

In [ ]:
# clean up the table name
conv_pct = conv_pct.relabel('1stDownConversion mean', 'rawConvPct')

# pivot to a better table form
conv_pct = conv_pct.pivot('YrdRegion', 'YardstoGo', values='rawConvPct', collect=np.mean)
conv_pct

In [ ]:
conv_pct.plot('YardstoGo', height=5, width=8)

ax = plt.gca()
ax.set_xlabel('YardstoGo')
ax.set_ylabel('ConvPct');

### B. Smoothed Expected Conversion Percentage

We'll use the curve fitting again to smooth out our Expected Conversion Percentage.

Our final model for Expected Conversion Percentage will be in `conv_pct_model`.

#### TO DO

In [ ]:
conv_pct_smooth = conv_pct.copy()
yards_to_go = conv_pct_smooth['YardstoGo']

# Create the smoothed model for each region of the field
for region in ['Inside10', '10to20', 'Beyond20']:
    conv_pct_region = conv_pct[region]
    
    # Fit a curve for conversion probability for a region using curve_fit
    # See the usage of curve_fit in the section on the expected possession value
    # use the default smoothness parameter
    smoothed_vals = ...

    conv_pct_smooth[region] = smoothed_vals

conv_pct_smooth

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

for region in ['Inside10', '10to20', 'Beyond20']:
    # plot the raw conversion percentage
    ax.plot(yards_to_go, conv_pct[region], ls=':', label='Raw Conv Pct ' + region)
    
    # plot the model conversion percentage
    ax.plot(yards_to_go, conv_pct_smooth[region], lw=3, label='xConvPct ' + region)

ax.legend(loc=1)
ax.set_ylabel('ConvPct');

#### Make the Model

In [ ]:
# Build the model for later use
conv_pct_model = {}
for region in ['Inside10', '10to20', 'Beyond20']:
    conv_pct_model[region] = make_model(conv_pct_smooth['YardstoGo'], conv_pct_smooth[region])
    
conv_pct_model

**Questions**

1. What is the likely reason that it makes a difference in conversion percentage where on the field the ball is?  For example, do you think it is easier or harder to convert a third down at your own 30 yard line versus at your opponent's 7 yard line?
2. The change in conversion percentage is about 10 percentage points between the middle of the field and inside the 10 yard line.  Can you quantify in points how meaningful those 10 percentage points are?

**Answers**

## Model 4: Field Goal Probability

We gather field goal results and compute the expected probability of success.

Raw field goal probability is a bit noisy as we have grown accustom to seeing so far.  We fit a curve but this time we ignore data observations beyond 63 yards since they are extra noisy.

### Field Goals

For field goal results, we just need FieldGoalDistance and FieldGoalResult and then to compute a 0-1 value representing the success.  This is straightforward.

In [ ]:
fgs = Table.read_table('data/nfl_fgs_2009_2016.csv', sep=',')
fgs.show(10)

### A. Field Goal Probability

#### TO DO

In [ ]:
# 1. Select the columns FieldGoalDistance and FieldGoalSuccess
# 2. Group by FieldGoalDistance 
# 3. Take the average on FieldGoalSuccess
fg_prob = fgs.\
    select('FieldGoalDistance', 'FieldGoalSuccess').\
    group('FieldGoalDistance', collect=np.mean)

In [ ]:
# clean up the column name
fg_prob = fg_prob.relabel('FieldGoalSuccess mean', 'rawFieldGoalProb')

# restrict to no longer than 63 yards
fg_prob = fg_prob.where('FieldGoalDistance', are.below_or_equal_to(63))
fg_prob

In [ ]:
fg_prob.scatter('FieldGoalDistance', select='rawFieldGoalProb', height=5, width=8)

### B. Smoothed Field Goal Probability

We can run another curve fit to get a nice estimate of field goal probability.

`fg_prob_model` will contain our field goal probability model.

#### TO DO

In [ ]:
fg_dist = fg_prob['FieldGoalDistance']
raw_fg_prob_values = fg_prob['rawFieldGoalProb']

# Fit a curve for FG probability using curve_fit
# See the usage of curve_fit in the section on the expected possession value
# use the default smoothness parameter
fg_prob_values = ...

# build the FG model
fg_prob['FieldGoalProb'] = fg_prob_values

fg_prob.scatter('FieldGoalDistance', select='rawFieldGoalProb', height=5, width=8)
ax = plt.gca()
plt.plot(fg_prob['FieldGoalDistance'], fg_prob['FieldGoalProb'], lw=3)
ax.grid(axis='y')

#### Make the Model

In [ ]:
fg_prob_model = make_model(fg_prob['FieldGoalDistance'], fg_prob['FieldGoalProb'])

**Question**

1. We are computing a baseline "average" model that can be used as a starting point.  Obviously the quality of the kicker and the field/weather conditions are incredibly important to kicking.  How would you adjust our model to account for these considerations?

**Answer**

## Decision Making

Let's put the models to work

### Extracting Model Values

First, let's show how we can extract outputs from the model based on a hypothetical situation.

It's 4th and 4 from the opponent's 42 yard line (this means `Yardline100` is 42).  What are the model outputs?

In [ ]:
yardline100 = 42
region = 'Beyond20'
yrdstogo = 4
fg_distance = yardline100 + 17

# Model values can be extracted easily
print(f"EPV at opponent 42 yardline: {epv_model[yardline100]:.2f}")
print(f"Exp Conv Pct beyond the 20 with 4 yards to go: {conv_pct_model[region][yrdstogo]:.2f}")
print(f"Exp Net Punt Dist from the 42: {punt_dist_model[yardline100]:.2f}")
print(f"Exp FG Pct from 45 yards: {fg_prob_model[fg_distance]:.2f}")

#### TO DO: Another Model Value Example

It's 4th and 1 from the opponent's 2 yard line.  What are the model outputs?  Note: Is the punt model valid here?

In [ ]:
yardline100 = 2
region = 'Inside10'
yrdstogo = 1
fg_distance = yardline100 + 17

...

### Expected Value of our Decisions

It's 4th and 1 from the opponent's 35 yard line.  What should we do?

In [ ]:
yardline100 = 35
region = 'Beyond20'
yrdstogo = 1
fg_distance = yardline100 + 17

#### TO DO: Expected value of going for it
1. First determine the value of a successful conversion.  Here are the rules for determining this value:
    + If the situation **is not** goal to go,  get the EPV of the yardline at the spot of the conversion.  Assume the down is converted with the smalled possible number of yards, ie. the yards to go.  So if it's 4th and 4 at the 42, the first down as at the 38.
    + If the situation **is** goal to go, then a 1st down conversion is a TD so take 7 minus the value of a kickoff.
2. Then determine the value of a failed conversion
    + Assume the conversion fails at the current yardline (not yards gained or lost on the failure).
    + Find the starting yardline for the opposing team by taking 100 minus the current yardline
    + Get the EPV for this starting position and multiply by -1 to get the value for _our team_ and not the opposing team.
3. Finally, compute the expected value using the conversion success probability
    + Weight the above expected values by the conversion success probability based on the region of the field

In [ ]:

ev_goforit = ...
ev_goforit  # 2.0800749018024116

#### TO DO:  Expected value of punting
1. Determine the expected net punt distance from the model.
2. Compute the expected value of the expected yardline after the punt
    + The expected yardline following the punt is the current yardline minus the expected punt distance.  So if we're at our own 30 yardline (Yardline100 = 70) and the punt distance is 42 yards, the expected yardline is 28.
    + Find the starting yardline for the opposing team by taking 100 minus the expected yardline after the punt
    + Get the EPV for the expected starting position and multiply by -1 to get the value for our team.

In [ ]:

ev_punt = ...
ev_punt  # 0.03172564552047145

#### TO DO:  Expected value of kicking
1. Compute the value of making the field goal: 3 minus the value of a kickoff, which is given by `ekv`.
2. Compute the value of missing the field goal.
    + Missing the kick means the ball is turned over at Yardline100 + 7
    + The starting position for the opposing team is 100 minus (Yardline100 + 7)
    + Get the EPV for the starting position
3. Compute the expected value using the field goal success probability
    + The kick distance is given by Yardline100 + 17
    + Use the kick distance to get the field goal success probability
    + Weight the two values in 1. and 2. by the field goal success probability

In [ ]:

ev_fg = ...
ev_fg  # 0.680632357737473 

#### Decision

We can print out the three expected values. 

You should get this:
```
EV going for it: 2.08
EV punting:      0.03
EV field goal:   0.68
```

In [ ]:
# print(f"EV going for it: {ev_goforit:.2f}")
# print(f"EV punting:      {ev_punt:.2f}")
# print(f"EV field goal:   {ev_fg:.2f}")

**Questions**

1. Which choice would you make?  Why?  What else would go into your decision?
2. How much extra is the decision worth in expected points?

**Answers**

## Visualizing Decisions

We can use a helper function which iterates over all the yardline and yards-to-go combinations to compute the decisions.

The function computes the following:
+ The best choice 
+ The expected value of the best choice
+ The second best choice
+ The _difference_ in expected value of the best choice over the second best choice

In [ ]:
data = compute_4thdownbot_data(
    ekv, epv_model, conv_pct_model, punt_dist_model, fg_prob_model)

yardline100 = 20
yrdstogo = 5
print(data[yrdstogo][yardline100])

### 4th Down Bot Decisions

We use a helper function to visualize the decision space.

In [ ]:
plot_4thdown_decision(data)

**Question**
1. If we just show the best decision choices, what are we missing in terms of information?  What else would you want to know regarding making any decisions?

**Answer**

### Second Choices

We can also look at the second best choices.

In [ ]:
plot_4thdown_second_choices(data)

**Question**
1. What appears to happen around the 37-38 yard line with respect to punting and kicking a field goal?

**Answer**

### Expected Values

It would be useful to know how many points we expect when making our decision.

In [ ]:
plot_4thdown_evs(data)

### Expected Value over Second Choice

Even more useful would be to know how many more points our top choice is worth relative to the next best choice.

In [ ]:
plot_4thdown_ev_over_second(data)

**Questions**
1. What is the significance of the bright yellow region around the 38 yard line?  What is it telling us about punting or kicking a field goal relative to trying to hold onto possession?
2. There is a huge region over which the 4th Down Bot says to go for it.  Does the increase in expected value from going for it seem particularly large over most of that region?

**Answers**

### Coaches' Choices

![coaches_decisions](coach_decisions.png "Coaches Decisions")

Compare the 4th Down Bot's decisions with the actual coaches decisions (courtesy of the NYT).  

**Question**
1. How does the previous plot showing the expected value over the second choice compare to the coaches' choices?

## Wrap Up

This notebook went over step by step how to piece together components using historical NFL play-by-play data to build a decision making 4th Down Bot.  The expected value analysis is very useful for beginning to think in a data-driven way.  Wrapping up, we finish with some general questions about expected value analysis and some considerations we might want to make.

In finance, there is a concept called _risk premium_ which refers to how much more return we demand for taking a riskier stock compared to a less risky stock.  That is, just because an stock has higher return, we will not necessarily take it due to the potential increase in risk.

Here's a simple example:  consider a game show where you may choose one of two doors which are hiding $\$$1,000 and $\$$0. Suppose I offer you $\$$500 instead of choosing a door.  Would you take the $\$$500 or choose a door?

The risk in this situation is the uncertainty of the payout and the possibility of ending up with $\$$0.  If you are unconcerned with risk, you view those options as equivalent.  If you are risk averse, you will opt for the $\$$500.  And if you are risk loving (ie. you derive benefit or utility or joy or something from taking risk) you will gamble on a door.

There is an amount called the _certainty equivalent_ where you would view that amount as equal to the gamble.  Since the expected value of the gamble is $\$$500, if you are risk averse the certainty equivalent is less than $\$$500.  It is the point at which below that amount you'd prefer the gamble and above that amount you'd prefer the certain payout.  Each person will have their own certainty equivalent.

The difference between the certainty equivalent and the expected value of the gamble is the risk premium.

The game show Deal or No Deal is an example of offering certainty amounts against gambles on large payouts.

**Questions**
1. How does punting represent a "certain" choice (or at least a less risky choice)?  
2. How does maximizing expected value ignore the risk in the decision?
   
   Consider this situation: we're on our own 10 yard line and its 4th and short.
   Some numbers for 4th and 1 from your own 10 yard line:
    + The expected value after conversion is about -0.09 points
    + The expected value after a failed conversion is -5.25 points
    + The conversion percentage is 69% so the overall expected value of going for it is -1.6
    + The expected value from punting is -2.3, .7 points lower than going for it.
   
   Overall, the EV for going for it is better.  But the downside of the failed conversion is high.  Is going for it a very risky decision based on those numbers?
   
4. Does maximizing expected points make sense throughout the game?  What about at the end of the game?


_Win Probability_

One possible alternative is to build a model for win probability given a game state.  The new NYT 4th Down Bot model does just this.  Using the win probability computation, a coach would make decisions that maximize win probability instead.  We saw something like this for bunting and stolen bases. 

Think of the classic Belichick choice against the Colts: go for it on 4th and short in your own territory at the end. [See here](https://fifthdown.blogs.nytimes.com/2009/11/16/defending-belichicks-fourth-down-decision/)

Here are some toy numbers.  Say it's 50-50 whether you make the conversion.  If you do make the conversion, you get a 100% win probability.  If you don't, you turn it over to Peyton Manning in his prime and now have a 20% win probability.  If you punt, you give the ball to Manning and its 60-40 in your favor.  Overall you have 60% win probability in each case.  
    
5. Does risk aversion still play a role when thinking about win probability?

_Ambiguity Aversion_

Risk averse people don't want to take risky bets even if they know the odds.  _Ambuigity averse_ people don't want to take bets they don't know the odds to  ([See here for an overview of ambiguity aversion](https://en.wikipedia.org/wiki/Ambiguity_aversion)).

So far, we've been quoting static numbers from models to build all our decisions.  We haven't pinned down how reliable or uncertain our models are.  

Let's reconsider Belichick's dilemma from 5.  We made up some numbers but they're not far off numbers other people have suggested.  So let's take our numbers as the output of the models.  An important question is how sure can we be these numbers are correct?  If the numbers aren't correct, do we want to take the gamble of going for it?  If we feel there is a lot of uncertainty in the model's output, we might be averse to this ambiguity and avoid taking the gamble.

6. How might a coach's decisions stem from both risk aversion and ambiguity aversion?
    
**Answers**